In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import soundfile as sf
from pathlib import Path
import re
import os
import json
from datetime import datetime,timedelta

## Annotation DB

### define paths for annotations and audio files

In [2]:
selection_path = Path('../../../../multispecies_dataset')
filelist = list(selection_path.glob('**/*.csv'))
# variables_names = [x.name.split('.')[0] for x in files_list]
filelist

[PosixPath('../../../../multispecies_dataset/HYDBBA105_Calls_2018_04_2kHz.csv'),
 PosixPath('../../../../multispecies_dataset/HYDBBA105_Calls_2018_04_EvenDays_MF.csv'),
 PosixPath('../../../../multispecies_dataset/HYDBBA105_Calls_2018_04_200Hz.csv'),
 PosixPath('../../../../multispecies_dataset/HYDBBA105_AcousticEvents_2018_04_EvenDays_AllRuns_v2.csv')]

### parse .csv file into pandas df

In [3]:
metadata =[]
metadata_dict = {}
for i,file in enumerate(filelist):
    dfTemp = pd.read_csv(file)
    print(file.name)
    print('Columns names')
    print(dfTemp.columns)
    print('Dataframe shape')
    print(dfTemp.shape)
#     dfTemp['filename'] = re.search(r'18\d{4,4}_\d{6,6}', file.as_posix()).group() 
#     dfTemp['StartMicInWater'] =np.amin(dfTemp['Begin Time (s)'])
#     dfTemp['EndMicInWater'] =np.amax(dfTemp['End Time (s)'])
    metadata.append(dfTemp)
    metadata_dict[file.name] = dfTemp

# metadata = pd.concat(metadata)
# metadata.rename(columns={'Begin Time (s)': 'begin_time', 'End Time (s)': 'end_time'}, inplace=True)

HYDBBA105_Calls_2018_04_2kHz.csv
Columns names
Index(['Id', 'UID', 'UTC', 'UTCMilliseconds', 'parentID', 'parentUID',
       'LongDataName', 'BinaryFile'],
      dtype='object')
Dataframe shape
(10424, 8)
HYDBBA105_Calls_2018_04_EvenDays_MF.csv
Columns names
Index(['Id', 'UID', 'UTC', 'UTCMilliseconds', 'PCLocalTime', 'PCTime',
       'ChannelBitmap', 'SequenceBitmap', 'parentID', 'parentUID',
       'LongDataName', 'BinaryFile'],
      dtype='object')
Dataframe shape
(600331, 12)
HYDBBA105_Calls_2018_04_200Hz.csv
Columns names
Index(['Id', 'UID', 'UTC', 'UTCMilliseconds', 'parentID', 'parentUID',
       'LongDataName', 'BinaryFile'],
      dtype='object')
Dataframe shape
(4802, 8)
HYDBBA105_AcousticEvents_2018_04_EvenDays_AllRuns_v2.csv
Columns names
Index(['Run', 'ID', 'PG_UID', 'UTC', 'UTCMilliseconds', 'EndTime', 'DataCount',
       'Analyst', 'Species_ID', 'Sound_type', 'Comments', 'Unnamed: 11'],
      dtype='object')
Dataframe shape
(136, 12)


In [4]:
Calls_2018_04_2kHz, Calls_2018_04_EvenDays_MF, Calls_2018_04_200Hz,EvenDays_AllRuns_v2 = metadata

In [5]:
pd.set_option('display.max_rows', None)

In [6]:
Calls_2018_04_200Hz.head()

,Id,UID,UTC,UTCMilliseconds,parentID,parentUID,LongDataName,BinaryFile
0,1,13000013,4/1/2018 12:05:53,395,1,1,"Whistle and Moan Detector, Contours ...",WhistlesMoans_Whistle_and_Moan_Detector_Contou...
1,2,13000015,4/1/2018 12:06:33,395,1,1,"Whistle and Moan Detector, Contours ...",WhistlesMoans_Whistle_and_Moan_Detector_Contou...
2,3,13000018,4/1/2018 12:06:52,275,1,1,"Whistle and Moan Detector, Contours ...",WhistlesMoans_Whistle_and_Moan_Detector_Contou...
3,4,13000022,4/1/2018 12:07:30,355,1,1,"Whistle and Moan Detector, Contours ...",WhistlesMoans_Whistle_and_Moan_Detector_Contou...
4,5,13000035,4/1/2018 12:11:33,235,1,1,"Whistle and Moan Detector, Contours ...",WhistlesMoans_Whistle_and_Moan_Detector_Contou...


In [7]:
np.unique(EvenDays_AllRuns_v2['Species_ID'])

array(['Delph sp.   ', 'Fin whale   ', 'Mn          ', 'Pm          ',
       'Pos Pm      ', 'Possible wh ', 'Ship        ', 'UNK Odonoto ',
       'Unknown sou ', 'Unknown wha '], dtype=object)

In [8]:
np.unique(EvenDays_AllRuns_v2['Sound_type'])

array(['BP          ', 'EC          ', 'EC, BP      ', 'EC, BP, WH  ',
       'EC, WH      ', 'Noise       ', 'Pulsed moan ', 'Pulses      ',
       'Song        ', 'Tonal calls ', 'Upsweeps    ', 'Varied      ',
       'WH          '], dtype=object)

In [12]:
EvenDays_AllRuns_v2[EvenDays_AllRuns_v2['PG_UID'] == 15]

,Run,ID,PG_UID,UTC,UTCMilliseconds,EndTime,DataCount,Analyst,Species_ID,Sound_type,Comments,Unnamed: 11
28,LF-200Hz,93,15,4/5/2018 13:52:16,435,4/5/2018 14:44:26,29,HMC,Possible wh,Pulses,"poss hybrid, 60 Hz pulses ...",NaN
58,LF-2kHz,66,15,4/10/2018 10:10:03,134,4/10/2018 10:14:32,388,HMC,Mn,Varied,NaN,NaN


In [13]:
Calls_2018_04_2kHz[Calls_2018_04_2kHz['parentUID']==15].loc[8771:9000]

,Id,UID,UTC,UTCMilliseconds,parentID,parentUID,LongDataName,BinaryFile
8771,8772,203000010,4/10/2018 10:10:03,134,15,15,"Whistle and Moan Detector, Contours ...",WhistlesMoans_Whistle_and_Moan_Detector_Contou...
8772,8773,203000011,4/10/2018 10:10:05,22,15,15,"Whistle and Moan Detector, Contours ...",WhistlesMoans_Whistle_and_Moan_Detector_Contou...
8773,8774,203000012,4/10/2018 10:10:11,687,15,15,"Whistle and Moan Detector, Contours ...",WhistlesMoans_Whistle_and_Moan_Detector_Contou...
8774,8775,203000013,4/10/2018 10:10:15,526,15,15,"Whistle and Moan Detector, Contours ...",WhistlesMoans_Whistle_and_Moan_Detector_Contou...
8775,8776,203000014,4/10/2018 10:10:16,871,15,15,"Whistle and Moan Detector, Contours ...",WhistlesMoans_Whistle_and_Moan_Detector_Contou...
8776,8777,203000015,4/10/2018 10:10:25,543,15,15,"Whistle and Moan Detector, Contours ...",WhistlesMoans_Whistle_and_Moan_Detector_Contou...
8777,8778,203000016,4/10/2018 10:10:25,55,15,15,"Whistle and Moan Detector, Contours ...",WhistlesMoans_Whistle_and_Moan_Detector_Contou...
8778,8779,203000017,4/10/2018 10:10:36,806,15,15,"Whistle and Moan Detector, Contours ...",WhistlesMoans_Whistle_and_Moan_Detector_Contou...
8779,8780,203000018,4/10/2018 10:10:40,318,15,15,"Whistle and Moan Detector, Contours ...",WhistlesMoans_Whistle_and_Moan_Detector_Contou...
8780,8781,203000020,4/10/2018 10:11:01,471,15,15,"Whistle and Moan Detector, Contours ...",WhistlesMoans_Whistle_and_Moan_Detector_Contou...


In [11]:
EvenDays_AllRuns_v2.head()

,Run,ID,PG_UID,UTC,UTCMilliseconds,EndTime,DataCount,Analyst,Species_ID,Sound_type,Comments,Unnamed: 11
0,LF-200Hz,79,1,4/1/2018 12:05:53,395,4/1/2018 12:25:32,6,HMC,Fin whale,Pulses,20 Hz ...,NaN
1,LF-200Hz,80,2,4/1/2018 17:22:58,355,4/1/2018 17:32:22,9,HMC,Fin whale,Pulses,NaN,NaN
2,LF-200Hz,81,3,4/1/2018 23:29:42,195,4/1/2018 23:29:42,25,HMC,Fin whale,Pulses,NaN,NaN
3,LF-200Hz,82,4,4/2/2018 9:12:04,475,4/2/2018 9:12:04,556,HMC,Fin whale,Pulses,NaN,NaN
4,LF-200Hz,83,5,4/2/2018 15:24:30,74,4/2/2018 15:32:47,13,HMC,Unknown wha,Pulses,"like Bp, but 50 hz (higher) ...",NaN


In [14]:
chosen_categories = ['Delph sp.   ', 'Fin whale   ', 'Mn          ']

In [16]:
filtered_categories_df = EvenDays_AllRuns_v2[EvenDays_AllRuns_v2['Species_ID'].isin(chosen_categories)]

In [17]:
filtered_categories_df[filtered_categories_df['PG_UID'] == 1]

,Run,ID,PG_UID,UTC,UTCMilliseconds,EndTime,DataCount,Analyst,Species_ID,Sound_type,Comments,Unnamed: 11
0,LF-200Hz,79,1,4/1/2018 12:05:53,395,4/1/2018 12:25:32,6,HMC,Fin whale,Pulses,20 Hz ...,NaN
6,LF-2kHz,52,1,4/2/2018 17:45:38,791,4/2/2018 18:04:23,42,HMC,Mn,Varied,Song & poss social ...,NaN


In [18]:
np.unique(filtered_categories_df['ID']).shape

(81,)

In [19]:
def df_freq_range_dict(x):
    return {
        'LF-200Hz': 'HYDBBA105_Calls_2018_04_200Hz.csv',
        'LF-2kHz': 'HYDBBA105_Calls_2018_04_2kHz.csv',
        'MF': 'HYDBBA105_Calls_2018_04_EvenDays_MF.csv'

    }.get(x, None)

In [43]:
def df_attach_wav_files(row):
    folder_prefix = '2018_04/OOI_105_'
    file_prefix = 'HYDBBA105OOI_'
#     date_time_obj = datetime. strptime(row['UTC'], '%Y-%m-%d %H:%M:%S.%f')
    format_1 = '%m/%d/%Y %H:%M:%S'
    format_2 = '%m/%d/%Y %H:%M'
    
    try:
        date_time_obj = datetime. strptime(row['UTC'], '%m/%d/%Y %H:%M:%S')
        date_time_obj = date_time_obj - timedelta(minutes=date_time_obj.minute % 10,
                                 seconds=date_time_obj.second,
                                 microseconds=date_time_obj.microsecond)
    #     '4/1/2018 12:05:53'
        begin_time = date_time_obj - timedelta(minutes=(date_time_obj.minute - (date_time_obj.minute % 10)),
                                 seconds=date_time_obj.second,
                                 microseconds=date_time_obj.microsecond)
        file_address = datetime.strftime(date_time_obj,folder_prefix + '%Y_%m_%d/' + file_prefix +'%Y%m%d-%H%M%S.wav')

    return file_address
    

In [44]:
def process_row(row, metadata_dict):
#     print(row['PG_UID'])
    freq_range_df = df_freq_range_dict(row['Run'])
    if freq_range_df is None:
        print('No such df!')
    else:
        chosen_freq_df = metadata_dict[freq_range_df]
        try:
            sub_df = chosen_freq_df[chosen_freq_df['parentUID'] == row['PG_UID']]
            sub_df = sub_df[['UID','UTC','UTCMilliseconds']]
            sub_df['Species_ID'] = row['Species_ID']
            sub_df['Sound_type'] = row['Sound_type']
            sub_df['PG_UID'] = row['PG_UID']
            sub_df['file'] = df_attach_wav_files(row)
        
    
    return sub_df

In [45]:
# for irow in filtered_categories_df[:2]:
# process_row(filtered_categories_df.iloc[0], metadata_dict)

In [46]:
filtered_categories_df.iloc[0]

Run                                                         LF-200Hz
ID                                                                79
PG_UID                                                             1
UTC                                                4/1/2018 12:05:53
UTCMilliseconds                                                  395
EndTime                                            4/1/2018 12:25:32
DataCount                                                          6
Analyst                                                   HMC       
Species_ID                                              Fin whale   
Sound_type                                              Pulses      
Comments           20 Hz                                         ...
Unnamed: 11                                                      NaN
Name: 0, dtype: object

In [47]:
# df_attach_wav_files(filtered_categories_df.iloc[0])

In [48]:
filtered_categories_df

,Run,ID,PG_UID,UTC,UTCMilliseconds,EndTime,DataCount,Analyst,Species_ID,Sound_type,Comments,Unnamed: 11
0,LF-200Hz,79,1,4/1/2018 12:05:53,395,4/1/2018 12:25:32,6,HMC,Fin whale,Pulses,20 Hz ...,NaN
1,LF-200Hz,80,2,4/1/2018 17:22:58,355,4/1/2018 17:32:22,9,HMC,Fin whale,Pulses,NaN,NaN
2,LF-200Hz,81,3,4/1/2018 23:29:42,195,4/1/2018 23:29:42,25,HMC,Fin whale,Pulses,NaN,NaN
3,LF-200Hz,82,4,4/2/2018 9:12:04,475,4/2/2018 9:12:04,556,HMC,Fin whale,Pulses,NaN,NaN
5,LF-200Hz,84,6,4/2/2018 17:08:15,875,4/2/2018 17:20:14,19,HMC,Fin whale,Pulses,NaN,NaN
6,LF-2kHz,52,1,4/2/2018 17:45:38,791,4/2/2018 18:04:23,42,HMC,Mn,Varied,Song & poss social ...,NaN
7,LF-2kHz,53,2,4/2/2018 19:12:25,471,4/2/2018 19:31:33,22,HMC,Mn,Varied,NaN,NaN
8,LF-2kHz,54,3,4/2/2018 21:39:17,151,4/2/2018 22:02:37,60,HMC,Mn,Varied,NaN,NaN
10,LF-2kHz,55,4,4/3/2018 0:14:06,463,4/3/2018 0:53:06,58,HMC,Mn,Varied,NaN,NaN
12,LF-2kHz,56,5,4/3/2018 4:18:21,22,4/3/2018 4:56:00,108,HMC,Mn,Varied,NaN,NaN


In [49]:
total_df = []
for _, row in filtered_categories_df.iterrows():
#     print(row)
    total_df.append(process_row(row, metadata_dict))

total_df = pd.concat(total_df)

4/4/2018 6:57


UnboundLocalError: local variable 'file_address' referenced before assignment

In [34]:
# total_df[total_df['PG_UID'] == 1].head(20)

In [35]:
total_df.iloc[5839,:]['UTC']

AttributeError: 'list' object has no attribute 'iloc'

In [ ]:
def df_attach_wav_files(row):
    folder_prefix = '2018_04/OOI_105_'
    file_prefix = 'HYDBBA105OOI_'
    date_time_obj = datetime. strptime(sample_row['UTC'], '%Y-%m-%d %H:%M:%S.%f')
    date_time_obj = date_time_obj - timedelta(minutes=date_time_obj.minute % 10,
                             seconds=date_time_obj.second,
                             microseconds=date_time_obj.microsecond)
    file_address = datetime.strftime(date_time_obj,folder_prefix + '%Y_%m_%d/' + file_prefix +'%Y%m%d-%H%M%S.wav')
    row[file_address]
    

In [32]:
sample_row = total_df.iloc[100000,:]

In [33]:
sample_row

UID                               94024171
UTC                2018-04-10 03:42:37.588
UTCMilliseconds                        588
Species_ID                    Delph sp.   
Sound_type                    WH          
Name: 146352, dtype: object

In [41]:
sample_row['UTC']

'2018-04-10 03:42:37.588'

In [84]:
sample_row['UTC']
date_time_obj = datetime. strptime(sample_row['UTC'], '%Y-%m-%d %H:%M:%S.%f')
date_time_obj

datetime.datetime(2018, 4, 10, 3, 42, 37, 588000)

In [85]:
date_time_obj = date_time_obj - timedelta(minutes=date_time_obj.minute % 10,
                             seconds=date_time_obj.second,
                             microseconds=date_time_obj.microsecond)

In [86]:
date_time_obj

datetime.datetime(2018, 4, 10, 3, 40)

In [87]:
date_time_obj

datetime.datetime(2018, 4, 10, 3, 40)

In [88]:
folder_prefix = '2018_04/OOI_105_'
file_prefix = 'HYDBBA105OOI_'

In [89]:
datetime.strftime(date_time_obj,'%H%M%S')

'034000'

In [92]:
datetime.strftime(date_time_obj,folder_prefix + '%Y_%m_%d/' + file_prefix +'%Y%m%d-%H%M%S.wav')

'2018_04/OOI_105_2018_04_10/HYDBBA105OOI_20180410-034000.wav'

In [45]:
# Calls_2018_04_2kHz, Calls_2018_04_EvenDays_MF, Calls_2018_04_200Hz

In [23]:
np.unique(filtered_categories_df['Run'])

array(['LF-200Hz', 'LF-2kHz', 'MF'], dtype=object)

In [24]:
print(Calls_2018_04_2kHz.shape,
Calls_2018_04_EvenDays_MF.shape,
Calls_2018_04_200Hz.shape)

(10424, 8) (600331, 12) (4802, 8)


In [25]:
metadata_dict.keys()

dict_keys(['HYDBBA105_Calls_2018_04_2kHz.csv', 'HYDBBA105_Calls_2018_04_EvenDays_MF.csv', 'HYDBBA105_Calls_2018_04_200Hz.csv', 'HYDBBA105_AcousticEvents_2018_04_EvenDays_AllRuns_v2.csv'])

In [26]:
Calls_2018_04_200Hz['parentUID'] == 

SyntaxError: invalid syntax (<ipython-input-26-69af40bc860e>, line 1)